# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-3.5-Turbo-1106` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-3.5 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-3.5-turbo-1106&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [7]:
from openai import OpenAI
client = OpenAI() 

In [2]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  max_tokens=4095,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

## 使用 LangChain 构造训练数据

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=1,
    max_tokens=4095,
    model_kwargs={
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

In [15]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [16]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [17]:
messages = [
    ("system", system_content),
    ("human", "{input}")
]

In [18]:
# ai_message = chat(messages)

# use LCEL
propmt = ChatPromptTemplate.from_messages(messages)
output = StrOutputParser()
chain = propmt | chat | output
result = chain.invoke({"input": raw_content})

### 解析 OpenAI GPT 生成的训练数据

In [19]:
result

'content:"蒙卦"\nsummary:"在《周易》中，蒙卦象征着教育启蒙的智慧，由两个异卦相叠构成：下卦为坎（水），上卦为艮（山）。艮卦代表停止，形象上如山；坎卦象征险阻，形如泉水，从山下流出。其意为，泉水从山中涌出，象征着初期的状态虽有险阻，但随时机把握，仍可顺利前进。\n\n蒙卦的核心哲学揭示了事物发展的初期阶段，通常处于蒙昧状态，因此教育显得尤为重要。教育的目标是培养学生纯正无邪的品德，这便是治蒙之道。虽然初始的蒙昧之象暗示着不明确和危险，但通过适时适当的行动，则可以启迪智慧，通达事物的真理。\n\n蒙卦作为《屯》卦之后的第二卦，《序卦》中提到：“物生必蒙，故受之以蒙”，这阐释了事物在幼稚阶段的自然状态，如同儿童的无知。进一步在《象》中解读，山下出泉，象征着启蒙，君子以果行育德，强调通过实践和行动来培育德行。"'

In [20]:
text = result

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 蒙卦
Summary: 在《周易》中，蒙卦象征着教育启蒙的智慧，由两个异卦相叠构成：下卦为坎（水），上卦为艮（山）。艮卦代表停止，形象上如山；坎卦象征险阻，形如泉水，从山下流出。其意为，泉水从山中涌出，象征着初期的状态虽有险阻，但随时机把握，仍可顺利前进。

蒙卦的核心哲学揭示了事物发展的初期阶段，通常处于蒙昧状态，因此教育显得尤为重要。教育的目标是培养学生纯正无邪的品德，这便是治蒙之道。虽然初始的蒙昧之象暗示着不明确和危险，但通过适时适当的行动，则可以启迪智慧，通达事物的真理。

蒙卦作为《屯》卦之后的第二卦，《序卦》中提到：“物生必蒙，故受之以蒙”，这阐释了事物在幼稚阶段的自然状态，如同儿童的无知。进一步在《象》中解读，山下出泉，象征着启蒙，君子以果行育德，强调通过实践和行动来培育德行。


### 持久化存储训练数据集文件

In [7]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [10]:
# by wy：优化+扩充提问模版
def generate_question_summary_pairs(content, summary):
    """
    生成30对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含30对提问和总结的列表。
    """
    # 30种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达其象征的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？",
        "{}的文化意义是什么？",
        "从周易的角度看，{}有何意义？",
        "在周易中，{}的象征作用是什么？",
        "{}对现代人有什么启发？",
        "{}象征着什么样的力量？",
        "{}的主要寓意是什么？",
        "{}在古代社会中的应用是什么？",
        "周易中的{}如何解读？",
        "{}的智慧是什么？",
        "从哲学角度，如何理解{}？",
        "解释一下{}的价值。",
        "{}与其他卦象的关系是什么？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    return [(question, summary) for question in questions]

In [21]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)
print(pairs)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


[('蒙卦代表什么？', '在《周易》中，蒙卦象征着教育启蒙的智慧，由两个异卦相叠构成：下卦为坎（水），上卦为艮（山）。艮卦代表停止，形象上如山；坎卦象征险阻，形如泉水，从山下流出。其意为，泉水从山中涌出，象征着初期的状态虽有险阻，但随时机把握，仍可顺利前进。\n\n蒙卦的核心哲学揭示了事物发展的初期阶段，通常处于蒙昧状态，因此教育显得尤为重要。教育的目标是培养学生纯正无邪的品德，这便是治蒙之道。虽然初始的蒙昧之象暗示着不明确和危险，但通过适时适当的行动，则可以启迪智慧，通达事物的真理。\n\n蒙卦作为《屯》卦之后的第二卦，《序卦》中提到：“物生必蒙，故受之以蒙”，这阐释了事物在幼稚阶段的自然状态，如同儿童的无知。进一步在《象》中解读，山下出泉，象征着启蒙，君子以果行育德，强调通过实践和行动来培育德行。'), ('周易中的蒙卦含义是什么？', '在《周易》中，蒙卦象征着教育启蒙的智慧，由两个异卦相叠构成：下卦为坎（水），上卦为艮（山）。艮卦代表停止，形象上如山；坎卦象征险阻，形如泉水，从山下流出。其意为，泉水从山中涌出，象征着初期的状态虽有险阻，但随时机把握，仍可顺利前进。\n\n蒙卦的核心哲学揭示了事物发展的初期阶段，通常处于蒙昧状态，因此教育显得尤为重要。教育的目标是培养学生纯正无邪的品德，这便是治蒙之道。虽然初始的蒙昧之象暗示着不明确和危险，但通过适时适当的行动，则可以启迪智慧，通达事物的真理。\n\n蒙卦作为《屯》卦之后的第二卦，《序卦》中提到：“物生必蒙，故受之以蒙”，这阐释了事物在幼稚阶段的自然状态，如同儿童的无知。进一步在《象》中解读，山下出泉，象征着启蒙，君子以果行育德，强调通过实践和行动来培育德行。'), ('请解释一下蒙卦。', '在《周易》中，蒙卦象征着教育启蒙的智慧，由两个异卦相叠构成：下卦为坎（水），上卦为艮（山）。艮卦代表停止，形象上如山；坎卦象征险阻，形如泉水，从山下流出。其意为，泉水从山中涌出，象征着初期的状态虽有险阻，但随时机把握，仍可顺利前进。\n\n蒙卦的核心哲学揭示了事物发展的初期阶段，通常处于蒙昧状态，因此教育显得尤为重要。教育的目标是培养学生纯正无邪的品德，这便是治蒙之道。虽然初始的蒙昧之象暗示着不明确和危险，但通过适时适当的行动，则可以启迪智慧，通达事物的真理。\n\n蒙卦作为《屯》卦之后的第二卦，《序卦》中提到：“物生

## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [12]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [13]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:3]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国

### 将以上的所有模块，整合到一起，自动化生成数据

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=1,
    max_tokens=4095,
    model_kwargs={
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)


def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    systemContent = """
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。"师"指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道："讼必有众起，故受之以师。师者，众也。"争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表"师"，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为"讼必有众起，故受之以师"。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """

    # 定义提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", systemContent),
        ("human", "{input}")
    ])

    # 定义输出解析器
    output_parser = StrOutputParser()

    # 构建LCEL链
    chain = prompt | chat | output_parser

    # 执行链并返回结果
    return chain.invoke({"input": raw_content})

In [23]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

content:"蒙卦"
summary:"在周易中，蒙卦是一个充满启示的卦象，由下卦坎（水）和上卦艮（山）组成。蒙的字义有“迷蒙”、“无知”，意指人在面对未知时的茫然与迷失。卦辞中提到，‘不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我’，反映出知识与智慧的双向关系，强调了教育与启蒙的重要性。

《象辞》解释道，上卦艮象征山，下卦坎象征泉，表示在山下有泉水涌出，象徵着艰难困境中蕴藏的宝贵智慧。君子应该以山泉的果敢和坚毅来培养自身的品德。

从传统解卦来看，蒙卦暗示在初期阶段可能会面临困惑与迷茫，但只要顺应时势，并能够虚心听取他人的意见与教诲，最终能够转危为机。在事业上，虽然面临混乱的局面，通过果敢行动和严格的教育，可以逐步扭转局面；而在经营上则要谨慎小心，树立良好的商业道德。对于婚恋关系，强调了考察对方品德的重要性，不得以物质为诱惑。

总之，蒙卦的核心哲学在于，面临困惑时应保持耐心，虚心向教导学习，待时而动，则终可通达。通过努力培养自身德行，将在运势、事业、财运、家庭等各方面迎来顺利的发展。"


In [24]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [25]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 蒙卦
Summary: 在周易中，蒙卦是一个充满启示的卦象，由下卦坎（水）和上卦艮（山）组成。蒙的字义有“迷蒙”、“无知”，意指人在面对未知时的茫然与迷失。卦辞中提到，‘不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我’，反映出知识与智慧的双向关系，强调了教育与启蒙的重要性。

《象辞》解释道，上卦艮象征山，下卦坎象征泉，表示在山下有泉水涌出，象徵着艰难困境中蕴藏的宝贵智慧。君子应该以山泉的果敢和坚毅来培养自身的品德。

从传统解卦来看，蒙卦暗示在初期阶段可能会面临困惑与迷茫，但只要顺应时势，并能够虚心听取他人的意见与教诲，最终能够转危为机。在事业上，虽然面临混乱的局面，通过果敢行动和严格的教育，可以逐步扭转局面；而在经营上则要谨慎小心，树立良好的商业道德。对于婚恋关系，强调了考察对方品德的重要性，不得以物质为诱惑。

总之，蒙卦的核心哲学在于，面临困惑时应保持耐心，虚心向教导学习，待时而动，则终可通达。通过努力培养自身德行，将在运势、事业、财运、家庭等各方面迎来顺利的发展。


In [26]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)


In [27]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦象征着通泰之意，其卦象由下卦坎（水）和上卦艮（山）组成。其主要含义为：不是我求于愚昧的幼稚者，而是幼稚者求于我。初次占卜能够得到启示，但如果轻慢再三占卜，则神灵将不再作答，然而此卦依然利于正当的卜问。

从象辞来看，上卦艮代表山，下卦坎象征泉水，山下涌出的泉水正体现了蒙卦的意象。君子应以此卦象为借鉴，勇敢果敢地行动，以培养自身的德行。

根据《断易天机》的解读，蒙卦代表着智慧未开、蒙昧无知，主指回归和犹豫。这是一个凶卦，暗示得卦者如同童蒙，不辨是非、迷失方向，但如能顺从贤师教诲、启迪聪明，则亨通无阻。

在北宋学者邵雍的阐释中，蒙卦充满了未见真理的困惑，缺乏果断和决定的能力。时运和财运方面，蒙卦暗示需积累德行、开创事业；矿产行业的生意则需果断以获吉。

在家庭和婚恋方面，君子安居乐业，婚姻之初需注重品德与相互包容，而不是追求物质利益。在事业上，此卦提醒需耐心应对初期的混乱，以坚毅的态度化解危机，强调基础教育和高尚道德的重要性。

总之，蒙卦的核心哲学在于启蒙和理解，初期可能会遇到迷惑和不安，但若保持耐心，从自身修养入手，顺应时势，则能够勇往直前，迎接顺利的未来。
Content: 屯卦
Summary: 在周易中，屯卦象征着困难与挑战，具有深刻的哲学内涵。其卦象由下卦震（雷）和上卦坎（云）组成，雷动于下，云行于上，体现了动与险并存的局势。屯卦寓意着万物初生时的艰难，强调先苦后甜的哲理，告诫人们在面对困境时应当坚韧不拔，付出努力，才能迎来转机与光明。

屯卦的传统解读认为，这个卦象表明处于困境之中，宜守不宜进，创业也许艰难，但是只要小心行事，适时把握机会，就能取得成功。在事业上，初期可能面临不利局面，但通过灵活应变、步步为营以及他人的援助，最终能够获得重大的成就。

感情方面，屯卦告诫求爱之人要坚持追求，尽管过程充满波折，忠诚和勇气将带来美满的婚姻。从财运上来看，创业之路较为坎坷，应保持坚定信念，克服表面困难，积极寻求突破，而在无力前进时则需退守以待良机。

总之，屯卦象征着万事初难，只要具备毅力和耐心，依靠内心的乐观精神，便可在挫折中迎接希望，最终走向成功的彼岸。
Content: 需卦
Summary: 需卦，在周易中具有丰富的象征意义，其卦象由上卦坎（水）和下卦乾（天）组成。需卦的含义为等待与期待，象征着云

### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 GPT-3.5-Turbo-1106 生成结果发现问题

In [30]:
def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    
    system_content = """
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:[卦名]
        summary:[内容]
        """
    
    # 定义提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_content),
        ("human", "{input}")
    ])

    # 定义输出解析器
    output_parser = StrOutputParser()

    # 构建LCEL链
    chain = prompt | chat | output_parser

    # 执行链并返回结果
    return chain.invoke({"input": raw_content})

In [31]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦是一个富含智慧的卦象，其卦形由下卦坎（水，象征泉）与上卦艮（山，象征阻止）组成。卦辞中强调，智慧的追求应该由幼稚愚昧的人向智慧者求教，而非相反。初次占筮时，如果存有敬畏之心，便能得到启示；若轻慢再三占筮，则知难而退。这一卦象寓意着审慎与果敢相结合，要从山泉的奋发中学习，以坚毅果敢的行动来培养自身的德行。

蒙卦呈现了一个初始混乱、迷惑不解的状态，显示出智慧仍在蒙昧之中。然而，如果能接受贤者的教诲，启迪聪明，最终便能通达亨通。此卦在各个方面的解读如下：在时运上需蓄积德行，待时而动；财运方面则适合矿山投资，但须果断；家庭与婚姻方面宜以品德为重，互相宽容理解；身体健康则要驱除邪热以保平安。

传统解卦认为，蒙卦象征着在险阻面前仍需前行、探索的精神，鼓励在迷惑中寻找方向，接受教育与成长，强调脚踏实地的重要性，若能在不断努力中迎接机遇，最终必能实现通达自由。总之，蒙卦警示人们在智慧尚未启迪之时，需耐心学习，谨慎行事，方能逆转困境，优化人生之旅。
Content: 屯卦
Summary: 屯卦，象征初创之时所面临的困难，构成于上卦坎（象征云）与下卦震（象征雷）之重叠，展现出云行于上、雷动于下的状态，代表了潜在的动荡与困难。此卦大吉大利，然而不利于出行，适合建立政权与封侯。

根据《象辞》，君子应以云雷之法治国，利用云的恩泽与雷的威严，显示出在艰难环境中所需的智慧与力量。庄严的初期困难表明，身处困境时应保持守势，多加努力排除障碍，以达到最终的成功。

北宋易学家邵雍提出，万物之初常伴艰辛，强调先劳后逸、苦尽甘来的道理。此卦的传统解读意识到凝聚人心的重要性，尤其在商业和事业发展的早期阶段，多有挫折需坚定信念，保持积极进取的态度。

在情感与婚恋方面，此卦暗示良好的关系往往经过磨难后才能收获美满的结果。面对初始困难，只有拥有坚韧不拔的毅力和乐观的处世态度，才能在未来走向成功。因此，在坑坎中不断前行，务必谨慎行事，珍惜身边的助力，方可趋吉避凶，最终实现理想。
Content: 需卦
Summary: 需卦在周易中，象征着期待与等待。该卦的上卦为坎（险水），下卦为乾（天），意指云聚于天上，待时降雨。此卦预示着抓住时机的重要性，虽有潜在的危险，但恰当的等待能够带来机遇和成功。

需卦的核心哲学是：在当前时机尚未成熟时，君子应保持耐心与